**Installing the Libraries, Models and Dataset**

In [ ]:
!pip install datasets python-Levenshtein transformers torch pandas

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Cipher-AI/Substitution-Cipher-Alphabet-Eng")
model = AutoModelForSeq2SeqLM.from_pretrained("Cipher-AI/Substitution-Cipher-Alphabet-Eng")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alphabet_model = model.to(device)
correction_model = AutoModelForSeq2SeqLM.from_pretrained("Cipher-AI/AutoCorrect-EN-v2").to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
from datasets import load_dataset

ds = load_dataset("agentlans/high-quality-english-sentences")

**Functions**

In [ ]:
from string import ascii_lowercase
import Levenshtein
import random

def similarity_percentage(s1, s2):
    distance = Levenshtein.distance(s1, s2)

    max_len = max(len(s1), len(s2))

    similarity = (1 - distance / max_len) * 100

    return similarity

def decode(cipher_text, key):
  decipher_map = {ascii_lowercase[i]: j for i, j in enumerate(key[:26])}
  decipher_map.update({ascii_lowercase[i].upper(): j.upper() for i, j in enumerate(key[:26])})
  ans = ''.join(map(lambda x: decipher_map[x] if x in decipher_map else x, cipher_text))
  return ans

def model_pass(model, input, max_length=256):
  inputs = tokenizer(input, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)
  outputs = model.generate(inputs["input_ids"], max_length=max_length)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

def decipher(cipher_text, key) -> str:
  decipher_map = {ascii_lowercase[i]: j for i, j in enumerate(key[0])}
  decipher_map.update({ascii_lowercase[i].upper(): j.upper() for i, j in enumerate(key[0])})

  result = ''.join(map(lambda x: decipher_map[x] if x in decipher_map else x, cipher_text[0]))

  return result

def cipher(plain_text) -> tuple[str, list]:
  alphabet_map = list(ascii_lowercase)
  random.shuffle(alphabet_map)
  alphabet_map = {i : j for i, j in zip(ascii_lowercase, alphabet_map)}

  alphabet_map.update({i.upper() : j.upper() for i, j in alphabet_map.items()})

  cipher_text = ''.join(map(lambda x: alphabet_map[x] if x in alphabet_map else x, plain_text))
  return cipher_text, alphabet_map

def correct_text(cipher_text, model_output):
  cipher_text = cipher_text.split(' ')
  model_output = model_output.split(' ')

  letter_map = {i: {j: 0 for j in ascii_lowercase} for i in ascii_lowercase}


  # Levenstein distance for lenghts of words
  n = len(cipher_text)
  m = len(model_output)

  i = 0
  j = 0
  dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

  for i in range(n + 1):
    dp[i][0] = i


  for j in range(m + 1):
    dp[0][j] = j

  for i in range(1, n + 1):
    for j in range(1, m + 1):
      if len(cipher_text[i - 1]) == len(model_output[j - 1]):
        dp[i][j] = dp[i - 1][j - 1]

      else:
        dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1

  i = n
  j = m
  while i > 0 and j > 0:

    before = min([(0, dp[i - 1][j - 1]), (1, dp[i - 1][j]), (2, dp[i][j - 1])], key=lambda x: x[1])
    match before[0]:
      case 0:
        if dp[i - 1][j - 1] == dp[i][j]:
          # If the same we add them to letter map
          cipher = cipher_text[i-1]
          model_o = model_output[j-1]

          for c_letter, m_letter in zip(cipher.lower(), model_o.lower()):
            if c_letter in letter_map and m_letter in letter_map[c_letter]:
              letter_map[c_letter][m_letter] += 1

        i = i - 1
        j = j - 1
      case 1:
        i = i - 1
      case 2:
        j = j - 1

  for letter in ascii_lowercase:
    letter_sum = sum(letter_map[letter].values())
    if letter_sum == 0:
      # That letter wasn't in the text
      letter_map[letter] = None
      continue

    # Sorted from most accuring to least
    letter_map[letter] = [(k, v / letter_sum) for k, v in sorted(letter_map[letter].items(), key=lambda item: item[1], reverse=True)]

  change_map = {
      i : None for i in ascii_lowercase
  }

  for i in range(len(ascii_lowercase)):
    for letter in ascii_lowercase:
      if letter_map[letter] is None:
        continue  # That letter wasn't in the text

      # If None then it didn't get substituted earlier
      map_letter = letter_map[letter][i][0]
      if (letter_map[letter][i][1] > 0 and (change_map[map_letter] is None
          or (change_map[map_letter][2] < letter_map[letter][i][1] and change_map[map_letter][1] >= i))):
        change_map[map_letter] = (letter, i, letter_map[letter][i][1])
        # Letter, iteration, percentage

  change_map = {i[1][0]: i[0] for i in change_map.items() if i[1] is not None}

  for letter in ascii_lowercase:
    if letter not in change_map:
      change_map[letter] = '.'


  # Add uppercases
  change_map.update(
    {
      i[0].upper() : i[1].upper() for i in change_map.items()
    }
  )

  new_text = []
  for cipher in cipher_text:
    new_word = ""
    for c_letter in cipher:
      if c_letter in change_map:
        new_word += change_map[c_letter]

      else:
        new_word += c_letter


    new_text.append(new_word)

  return ' '.join(new_text)

def crack_sub(cipher_text):
  output = model_pass(alphabet_model, cipher_text, 26)
  decoded = decode(cipher_text, output)
  second_pass = model_pass(correction_model, decoded, len(decoded))
  second_text = correct_text(cipher_text, second_pass)

  return second_text



Improvements that can be made to correct_text():
- Only a single letter maps to a letter [x]
- A system for proccessing inputs with different world lengths [ ]
- A system for proccessing words of different lenght [ ]




**Benchmark for merged model**

In [ ]:
import csv
from itertools import islice

same = 0
all = 0

start = 0
end = 1000

with open("/content/test_ans.csv", 'w') as model_outputf:
  model_output = csv.writer(model_outputf)

  model_output.writerow(["cipher_text", "plain_text", "decoded", "second_pass", "final_text", "third_pass", "Decoded Similarity", "Second pass Similarity", "Second text Similarity", "Third pass similarity"])
  for plain_text in islice(ds['test']['text'], start, end):
    cipher_text, _ = cipher(plain_text)

    output = model_pass(alphabet_model, cipher_text, 26)
    decoded = decode(cipher_text, output)
    second_pass = model_pass(correction_model, decoded, len(decoded))
    second_text = correct_text(cipher_text, second_pass)
    third_pass = model_pass(correction_model, second_text, len(decoded))

    model_output.writerow([cipher_text, plain_text, decoded, second_pass, second_text, third_pass, similarity_percentage(decoded, plain_text), similarity_percentage(second_pass, plain_text), similarity_percentage(second_text, plain_text), similarity_percentage(third_pass, plain_text)])

    same += Levenshtein.distance(third_pass, plain_text)

    all += len(plain_text)

  print(1 - same/all)

0.9196303126097647


**TEST DATA AVERAGE FOR ALPHABET MODEL**

In [ ]:
import csv

same = 0
all = 0
with open("/content/test_cipher.csv", 'r') as cipher_file, open("/content/test_plain.csv", 'r') as plain_file, open("/content/test_ans.csv", 'w') as ans_file:
    cipher_texts = csv.reader(cipher_file)
    plain_texts = csv.reader(plain_file)
    ans_writer = csv.writer(ans_file)
    ans_writer.writerow(["decoded_text", "ans", "plain_text"])
    num = 0
    for cipher_text, plain_text in zip(cipher_texts, plain_texts):
        if(num == 1000):
          break
        num += 1
        cipher_text = cipher_text[0]
        plain_text = plain_text[0]

        inputs = tokenizer(cipher_text, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)

        outputs = model.generate(inputs["input_ids"], max_length=26)

        decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        decipher_map = {ascii_lowercase[i]: j for i, j in enumerate(decoded_text[:26])}
        decipher_map.update({ascii_lowercase[i].upper(): j.upper() for i, j in enumerate(decoded_text[:26])})
        ans = ''.join(map(lambda x: decipher_map[x] if x in decipher_map else x, cipher_text))
        #print("answer : " + ans)


        same += Levenshtein.distance(ans, plain_text)
        ans_writer.writerow([decoded_text, ans, plain_text])

        all += len(plain_text)
    print(1 - same/all)

0.7356574171642665


**AVERAGE IN TEST DATA WITH AUTOCORRECT FOR ALPHABET MODEL**

In [ ]:
import csv

same = 0
all = 0
with open("/content/test_cipher.csv", 'r') as cipher_file, open("/content/test_plain.csv", 'r') as plain_file:
    cipher_texts = csv.reader(cipher_file)
    plain_texts = csv.reader(plain_file)
    num = 0
    for cipher_text, plain_text in zip(cipher_texts, plain_texts):
        if(num == 100):
          break
        num += 1
        cipher_text = cipher_text[0]
        plain_text = plain_text[0]

        inputs = tokenizer(cipher_text, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)

        outputs = model.generate(inputs["input_ids"], max_length=26)

        decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        decipher_map = {ascii_lowercase[i]: j for i, j in enumerate(decoded_text[:26])}
        decipher_map.update({ascii_lowercase[i].upper(): j.upper() for i, j in enumerate(decoded_text[:26])})
        ans = ''.join(map(lambda x: decipher_map[x] if x in decipher_map else x, cipher_text))
        ans = spell(ans)
        #print("answer : " + ans)

        same += Levenshtein.distance(ans, plain_text)

        all += len(plain_text)
    print(1 - same/all)

0.7542468008680685


**AVERAGE ON TEST DATA FOR TEXT MODEL**

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained("Cipher-AI/Substitution-Cipher-Text-Eng")
model2 = AutoModelForSeq2SeqLM.from_pretrained("Cipher-AI/Substitution-Cipher-Text-Eng").to(device)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
import csv

same = 0
all = 0
with open("/content/test_cipher.csv", 'r') as cipher_file, open("/content/test_plain.csv", 'r') as plain_file:
    cipher_texts = csv.reader(cipher_file)
    plain_texts = csv.reader(plain_file)
    num = 0
    for cipher_text, plain_text in zip(cipher_texts, plain_texts):
        if(num == 1000):
          break
        num += 1
        cipher_text = cipher_text[0]
        plain_text = plain_text[0]

        inputs = tokenizer2(cipher_text, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)

        outputs = model2.generate(inputs["input_ids"], max_length=256)

        decoded_text = tokenizer2.decode(outputs[0], skip_special_tokens=True)
        #print("answer : " + ans)

        same += Levenshtein.distance(plain_text, decoded_text)

        all += len(plain_text)
    print(1 - same/all)

0.7251419622994966


**AVERAGE ON TEST DATA WITH AUTOCORRECT FOR TEXT MODEL**

In [ ]:
import csv

same = 0
all = 0
with open("/content/test_cipher.csv", 'r') as cipher_file, open("/content/test_plain.csv", 'r') as plain_file:
    cipher_texts = csv.reader(cipher_file)
    plain_texts = csv.reader(plain_file)
    num = 0
    for cipher_text, plain_text in zip(cipher_texts, plain_texts):
        if(num == 100):
          break
        num += 1
        cipher_text = cipher_text[0]
        plain_text = plain_text[0]

        inputs = tokenizer2(cipher_text, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)

        outputs = model2.generate(inputs["input_ids"], max_length=256)

        decoded_text = tokenizer2.decode(outputs[0], skip_special_tokens=True)
        decoded_text = spell(decoded_text)
        #print("answer : " + ans)

        same += Levenshtein.distance(plain_text, decoded_text)

        all += len(plain_text)
    print(1 - same/all)

0.7193743919778492
